In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nepali-dataset/npi.txt
/kaggle/input/nepali-dataset/_about.txt


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input,LSTM,Dense
data_path='/kaggle/input/nepali-dataset/npi.txt'
batch_size=8
epochs=100
latent_dim=512


In [198]:

with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')

In [199]:

texts=[]
for line in lines:
    texts.append(line.split('\t')[0:2])

In [200]:
input_texts=[]
output_texts=[]
input_characters=set()
output_characters=set()
for i in range(len(texts)-1):
    input_texts.append(texts[i][0])
    output_texts.append(texts[i][1])
    for char in input_texts[i]:
        if char not in input_characters:
            input_characters.add(char)
    for char in output_texts[i]:
            if char not in output_characters:
                output_characters.add(char)

In [201]:
print(len(input_texts))
print(len(output_texts))
print(len(output_characters))
print(len(output_characters))

2405
2405
76
76


In [202]:
input_characters=sorted(list(input_characters))
output_characters=sorted(list(output_characters))
num_encoders_token=len(input_characters)
num_decoders_token=len(output_characters)

In [203]:
max_encoders_length=max([len(txt) for txt in input_texts])
max_decoders_length=max([len(txt) for txt in output_texts])

In [204]:
print("num of samples",len(input_texts))
print("number of unique input tokens",num_encoders_token)
print("number of unique output tokens",num_decoders_token)

print(max_encoders_length)
print(max_decoders_length)

num of samples 2405
number of unique input tokens 67
number of unique output tokens 76
122
102


In [205]:
input_token_index=dict(
    [(char,i) for i,char in enumerate(input_characters)]
)
output_token_index=dict(
    [(char,i) for i,char in enumerate(output_characters)]
)

In [206]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoders_length,num_encoders_token),dtype='float32')

decoder_input_data=np.zeros(
    (len(output_texts),max_decoders_length,num_decoders_token),dtype='float32')

decoder_target_data=np.zeros(
    (len(output_texts),max_decoders_length,num_decoders_token)
,dtype='float32')

In [207]:
for i, (input_text, output_text) in enumerate(zip(input_texts, output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1
    for t, char in enumerate(output_text):
        decoder_input_data[i, t, output_token_index[char]] = 1
        if t > 0:     
            decoder_target_data[i, t - 1, output_token_index[char]] = 1
decoder_input_data[i, t + 1:, output_token_index[' ']] = 1
decoder_target_data[i, t:, output_token_index[' ']] = 1


In [208]:
print(decoder_input_data[0].shape)
print(encoder_input_data[0].shape)


(102, 76)
(122, 67)


In [235]:
encoder_inputs=Input(shape=(None,num_encoders_token))
encoder=LSTM(latent_dim,return_state=True)

encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]



In [236]:
decoder_inputs=Input(shape=(None,num_decoders_token))
decoder_lstm=LSTM(latent_dim,return_state=True,return_sequences=True)

decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoders_token,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [258]:
from keras.layers import LSTM, Input, Dense,GRU,Dropout,LayerNormalization

# Encoder
encoder_inputs = Input(shape=(None, num_encoders_token))
encoder_lstm_1 = GRU(latent_dim, return_sequences=True, return_state=True)
encoder_outputs_1, state_h_1 = encoder_lstm_1(encoder_inputs)
encoder_outputs_1 = Dropout(0.2)(encoder_outputs_1)
encoder_outputs_1 = LayerNormalization()(encoder_outputs_1)


encoder_lstm_2 = GRU(latent_dim,return_sequences=True,  return_state=True)
encoder_outputs_2, state_h_2 = encoder_lstm_2(encoder_outputs_1)
encoder_outputs_2 = Dropout(0.2)(encoder_outputs_2)
encoder_outputs_2 = LayerNormalization()(encoder_outputs_2)



encoder_lstm_3= GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder_lstm_3(encoder_outputs_2)
encoder_outputs = Dropout(0.2)(encoder_outputs)
encoder_outputs = LayerNormalization()(encoder_outputs)

encoder_states = state_h

# Decoder
decoder_inputs = Input(shape=(None, num_decoders_token))
decoder_lstm_1 = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs_1, _ = decoder_lstm_1(decoder_inputs, initial_state=encoder_states)
decoder_outputs_1 = Dropout(0.2)(decoder_outputs_1)
decoder_outputs_1= LayerNormalization()(decoder_outputs_1)


decoder_lstm_2 = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs_2, _ = decoder_lstm_2(decoder_outputs_1)
decoder_outputs_2 = Dropout(0.2)(decoder_outputs_2)
decoder_outputs_2= LayerNormalization()(decoder_outputs_2)


decoder_lstm_2 = GRU(latent_dim,return_state=True)
decoder_outputs, _ = decoder_lstm_2(decoder_outputs_2)
decoder_outputs = Dropout(0.2)(decoder_outputs)
decoder_outputs= LayerNormalization()(decoder_outputs)



decoder_dense = Dense(num_decoders_token, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [259]:
from keras.optimizers import Adam
optimizer=Adam(learning_rate=0.0001)
model.summary()
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data,decoder_input_data],
                  decoder_target_data,
                  epochs=epochs,
                  verbose=1,
                  validation_split=0.2
                 )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, 67)]           0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None, 76)]           0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 200),                214400    ['input_2[0][0]']             
                              (None, 200),                                                        
                              (None, 200)]                                                        
                                                                                              

In [218]:
model.save("predmodel.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [287]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))

    # Sampling loop for a batch of sequences (assuming batch size 1).
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        # Predict the next token
        output_tokens, states_value = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

    return decoded_sentence


In [288]:
for seq_index in range(10):
    input_seq=encoder_input_data[seq_index:seq_index+1]
    decoded_sentence=decode_sequence(input_seq)
    print('--')
    print('Input Sentence ',input_texts[seq_index])
    print('Decoded Sentence ', decoded_sentence)


1/1 [==============================] - 1s 885ms/step


IndexError: index 1 is out of bounds for axis 0 with size 1

In [270]:
print(tokenizer.word_index)

{' ': 1, 'e': 2, 't': 3, 'o': 4, 'ा': 5, 'a': 6, '्': 7, 'i': 8, 'न': 9, 'म': 10, 'n': 11, 's': 12, 'र': 13, 'क': 14, 'h': 15, 'r': 16, 'ो': 17, 'े': 18, 'ल': 19, 'ि': 20, 'm': 21, 'd': 22, 'ह': 23, '.': 24, '।': 25, 'l': 26, 'स': 27, 'ु': 28, 'y': 29, 'छ': 30, 'w': 31, 'u': 32, 'त': 33, 'य': 34, 'ट': 35, 'प': 36, 'ी': 37, 'ग': 38, 'g': 39, 'c': 40, 'ै': 41, 'ई': 42, '?': 43, "'": 44, 'द': 45, 'ब': 46, 'f': 47, 'b': 48, 'k': 49, 'p': 50, 'ँ': 51, 'v': 52, 'भ': 53, 'ज': 54, 'च': 55, 'ए': 56, 'उ': 57, 'थ': 58, 'व': 59, 'ख': 60, 'ं': 61, 'अ': 62, 'ध': 63, 'फ': 64, 'ू': 65, 'आ': 66, ',': 67, 'श': 68, 'ड': 69, 'ौ': 70, 'इ': 71, 'ष': 72, 'ठ': 73, 'x': 74, 'ण': 75, 'झ': 76, '"': 77, 'ढ': 78, 'घ': 79, 'ृ': 80, 'j': 81, 'z': 82, 'ऊ': 83, 'q': 84, '!': 85, 'ङ': 86, '0': 87, '-': 88, 'औ': 89, '१': 90, '3': 91, 'ओ': 92, '३': 93, '०': 94, 'ः': 95, '1': 96, 'ञ': 97, ':': 98, '2': 99, '\u200b': 100, 'ऋ': 101, '9': 102, '९': 103, '4': 104, '२': 105, '४': 106, '€': 107, '\u200d': 108}
